## Calculate Interpretability Data!

## Prep for analysis

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import tensorflow as tf
import random 
from datetime import datetime
import pandas as pd
import os 

import sys
sys.path.append('/glade/work/kjmayer/research/catalyst/ENSOvsMJO/utils/')
# sys.path.append('/glade/u/home/wchapman/ENSOvsMJO/utils/')
from exp_hp import get_hp
from trainGordon_utils import subset, build_model, fullmodel, scheduler, plot_results, adjust_spines
from dataprep_utils import get_testing
sys.path.append('/glade/work/kjmayer/research/catalyst/ENSOvsMJO/interpret/')
# sys.path.append('/glade/u/home/wchapman/ENSOvsMJO/interpret/')
from Gordon_interp import getoutputvecs, confvacc, iconfcorr


# import importlib
# importlib.reload(sys.modules["Gordon_interp"])
# from Gordon_interp import getoutputvecs, confvacc, iconfcorr
# importlib.reload(sys.modules["trainGordon_utils"])
# from trainGordon_utils import subset, build_model, fullmodel, scheduler, plot_results

import matplotlib as mpl
import matplotlib.pyplot as plt

plt.rc('text',usetex=True)
plt.rcParams['font.family']='sans-serif'
plt.rcParams['font.sans-serif']=['Verdana']
plt.rcParams.update({'font.size': 15})
def adjust_spines(ax, spines):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 5))
        else:
            spine.set_color('none')
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        ax.yaxis.set_ticks([])
    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
            ax.xaxis.set_ticks([])
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi'] = 150
dpiFig = 300.

2023-11-16 09:19:45.247372: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 09:19:46.963968: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Load network:

In [2]:
MODEL_DIR = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/models/'
# MODEL_DIR = '/glade/scratch/wchapman/ENSOmjo_ML_models/saved_models/'
EXP_NAME = 'default'
hps = get_hp(EXP_NAME)
# variables:
DROPOUT_RATE = hps['DROPOUT_RATE']

MODELNAME1 = 'ENSO'
RIDGE1 = hps['RIDGE1']
HIDDENS1 = hps['HIDDENS1']

MODELNAME2 = 'MJO'
RIDGE2 = hps['RIDGE2']
HIDDENS2 = hps['HIDDENS2']

BATCH_SIZE = hps['BATCH_SIZE']
N_EPOCHS = 10000
PATIENCE = hps['PATIENCE'] # number of epochs of no "improvement" before training is stopped
LR = hps['LR'] # learning rate


LEADS = np.arange(7,31)
AVGS = np.arange(2,32)
SEEDS = np.arange(1,6)

SAVE = True

### Analyze Data

In [3]:
# model raw predictions
# confidence vs accuracy
# model contribution fractions

for l in LEADS[:1]:
    for a in AVGS[:1]:
        print('LEAD: '+str(l)+'\nAVG: '+str(a))
        
        #check if files already exist:
        ddir_save = '/glade/scratch/kjmayer/DATA/ENSOvsMJO/data/'
        finame_confvsacc = 'confvsacc_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
        if not os.path.isfile(ddir_save+finame_confvsacc):
            # print('load testing data')
            X1test, X2test, Ytest = get_testing(N_z500runmean=a,
                                                LEAD=l)

            INPUT_SHAPE1 = np.shape(X1test)[1:][0]
            INPUT_SHAPE2 = np.shape(X2test)[1:][0]

            confvsacc = np.zeros(shape=(len(SEEDS),100))

            model1_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))
            model2_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))
            model_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))
            
            model1_all_fracpred = np.zeros(shape=(len(SEEDS)))
            model2_all_fracpred = np.zeros(shape=(len(SEEDS)))
            model12_all_fracpred = np.zeros(shape=(len(SEEDS)))
            
            model1_allcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            model2_allcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            model12_allcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            
            model1_conf_fracpred = np.zeros(shape=(len(SEEDS)))
            model2_conf_fracpred = np.zeros(shape=(len(SEEDS)))
            model12_conf_fracpred = np.zeros(shape=(len(SEEDS)))
            
            model1_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            model2_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            model12_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))
            
            
            for s in SEEDS:
                # ENSO MODEL
                model1, input1 = build_model(s,
                                             DROPOUT_RATE,
                                             RIDGE1,
                                             HIDDENS1,
                                             INPUT_SHAPE1,
                                             MODELNAME1)
                # MJO MODEL
                model2, input2 = build_model(s,
                                             DROPOUT_RATE,
                                             RIDGE2,
                                             HIDDENS2,
                                             INPUT_SHAPE2,
                                             MODELNAME2)   
                # COMBINE ENSO & MJO MODEL
                model = fullmodel(model1, model2,
                                  input1, input2,
                                  s)

                MODEL_FINAME = 'LEAD_'+str(l)+'_AVG_'+str(a)+'__0000'+str(s)+'.h5'
                model.load_weights(MODEL_DIR+MODEL_FINAME)

                model_rawpreds[s-1] = model.predict((X1test,X2test))                
                
                conf    = np.max(model_rawpreds[s-1],axis=-1)
                predval = np.argmax(model_rawpreds[s-1],axis=-1)
                
                # ------- confident predictions --------------------------------------------------------
                per = 80
                conf_thresh = np.percentile(conf,q=per)
                # -------- confident [i_conf_predval] --------
                i_conf_predval = np.where(conf > conf_thresh)[0]
                
                
                # ----- confidence vs accuracy for all seeds: -------------------------------------------
                confvsacc[s-1], _, _ = confvacc(confval = conf,
                                              predval = predval,
                                              Ytest   = Ytest)

                # ----- model contribution: ------------------------------------------------------------
                model1_rawpreds[s-1], model2_rawpreds[s-1] = getoutputvecs(model,
                                                                       model1,
                                                                       model2,
                                                                       X1test,
                                                                       X2test)
                # model X winning class (model X output * weight)
                model1pred = np.argmax(model1_rawpreds[s-1],axis=1)
                model2pred = np.argmax(model2_rawpreds[s-1],axis=1)

                # --------------------------------------------------------------------------------------
                # ---------- all predictions contribution: ---------------------------------------------
                # --------------------------------------------------------------------------------------
                i_model1_samefinalpred = model1pred==predval
                i_model2_samefinalpred = model2pred==predval
                
                # terminology: "win" = modelX prediction is also full model prediction
                # model X same as final prediction & model ~X does not have that prediction
                i_model1win = i_model1_samefinalpred & ~i_model2_samefinalpred
                i_model2win = i_model2_samefinalpred & ~i_model1_samefinalpred
                # model 1&2 have same final prediction
                i_model12win = i_model1_samefinalpred & i_model2_samefinalpred
                
                # number of predictions of model(X) predicted class that is also full model prediction
                n_model1win_predval = model1pred[i_model1win].shape[0]
                n_model2win_predval = model2pred[i_model2win].shape[0]
                n_model12win_predval = predval[i_model12win].shape[0]
                
                # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
                if predval.shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
                    # print('SEED: '+str(s))
                    # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
                    model1_all_fracpred[s-1]  = (n_model1win_predval/predval.shape[0])
                    model2_all_fracpred[s-1]  = (n_model2win_predval/predval.shape[0])
                    model12_all_fracpred[s-1] = (n_model12win_predval/predval.shape[0])
                
                
                
                # --------------------------------------------------------------------------------------
                #  ---------- all correct predictions contribution:  -----------------------------------
                # --------------------------------------------------------------------------------------
                # Where ENSO/MJO/final model (model 1/model 2/total) are correct:
                i_model1corr = model1pred==Ytest
                i_model2corr = model2pred==Ytest
                i_modelcorr  = predval==Ytest

                # terminology: "win" = modelX prediction is also (correct) full model prediction
                # model X correct & model correct (model ~X not correct)
                i_model1win = i_model1corr & i_modelcorr & ~i_model2corr
                i_model2win = i_model2corr & i_modelcorr & ~i_model1corr
                # model 1&2 correct & model correct
                i_model12win = i_model1corr & i_model2corr & i_modelcorr

                # For correct predictions: model(X) values & predicted class when also full model prediction
                model1win_contribution = model1_rawpreds[s-1][i_model1win]
                model1win_predval = model1pred[i_model1win]

                model2win_contribution = model2_rawpreds[s-1][i_model2win]
                model2win_predval = model2pred[i_model2win]

                model12win_predval = model_rawpreds[s-1][i_model12win]

                # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
                if model_rawpreds[s-1][i_modelcorr].shape[0] == model1win_predval.shape[0] + model2win_predval.shape[0] + model12win_predval.shape[0]:
                    # print('SEED: '+str(s))
                    # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
                    model1_allcorr_fracpred[s-1]  = (model1win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])
                    model2_allcorr_fracpred[s-1]  = (model2win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])
                    model12_allcorr_fracpred[s-1] = (model12win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])        

            
            
                # --------------------------------------------------------------------------------------
                # --------- confident predictions contribution: ----------------------------------------
                # --------------------------------------------------------------------------------------
                i_model1_samefinalconfpred = model1pred[i_conf_predval]==predval[i_conf_predval]
                i_model2_samefinalconfpred = model2pred[i_conf_predval]==predval[i_conf_predval]
                
                # terminology: "win" = modelX prediction is also full model prediction
                # model X same as final prediction & model ~X does not have that prediction
                i_model1win = i_model1_samefinalconfpred & ~i_model2_samefinalconfpred
                i_model2win = i_model2_samefinalconfpred & ~i_model1_samefinalconfpred
                # model 1&2 have same final prediction
                i_model12win = i_model1_samefinalconfpred & i_model2_samefinalconfpred
                
                
                # number of predictions of model(X) predicted class that is also full model prediction
                n_model1win_predval = model1pred[i_conf_predval][i_model1win].shape[0]
                n_model2win_predval = model2pred[i_conf_predval][i_model2win].shape[0]
                n_model12win_predval = predval[i_conf_predval][i_model12win].shape[0]

                
                # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
                if predval[i_conf_predval].shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
                    # print('SEED: '+str(s))
                    # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
                    model1_conf_fracpred[s-1]  = (n_model1win_predval/predval[i_conf_predval].shape[0])
                    model2_conf_fracpred[s-1]  = (n_model2win_predval/predval[i_conf_predval].shape[0])
                    model12_conf_fracpred[s-1] = (n_model12win_predval/predval[i_conf_predval].shape[0]) 
                
                
                
                # --------------------------------------------------------------------------------------
                # --------- confident & correct predictions contribution:-------------------------------
                # --------------------------------------------------------------------------------------
                # Where ENSO/MJO/final model (model 1/model 2/total) are correct
                i_model1_confcorr = model1pred[i_conf_predval]==Ytest[i_conf_predval]
                i_model2_confcorr = model2pred[i_conf_predval]==Ytest[i_conf_predval]
                i_model_confcorr  = predval[i_conf_predval]==Ytest[i_conf_predval]

                # terminology: "win" = modelX prediction is also (correct) full model prediction
                # model X correct & model correct (model ~X not correct)
                i_model1win = i_model1_confcorr & i_model_confcorr & ~i_model2_confcorr
                i_model2win = i_model2_confcorr & i_model_confcorr & ~i_model1_confcorr
                # model 1&2 correct & model correct
                i_model12win = i_model1_confcorr & i_model2_confcorr & i_model_confcorr

                # For correct predictions: model(X) values & predicted class when also full model prediction
                n_model1win_predval = model1pred[i_conf_predval][i_model1win].shape[0]
                n_model2win_predval = model2pred[i_conf_predval][i_model2win].shape[0]
                n_model12win_predval = predval[i_conf_predval][i_model12win].shape[0]
                
                
                # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
                if predval[i_conf_predval][i_model_confcorr].shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
                    # print('SEED: '+str(s))
                    # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
                    model1_confcorr_fracpred[s-1]  = (n_model1win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])
                    model2_confcorr_fracpred[s-1]  = (n_model2win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])
                    model12_confcorr_fracpred[s-1] = (n_model12win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])
            
            if SAVE:
                print('saving')
                ddir_save = '/glade/scratch/kjmayer/DATA/ENSOvsMJO/data/'
                
                finame_confvsacc = 'confvsacc_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_confvsacc, confvsacc)

                finame_rawpred = 'model1_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_rawpred, model1_rawpreds)
                finame_rawpred = 'model2_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_rawpred, model2_rawpreds)
                finame_rawpred = 'model_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_rawpred, model_rawpreds)

                finame_fracpred = 'model1_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model1_allcorr_fracpred)
                finame_fracpred = 'model2_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model2_allcorr_fracpred)
                finame_fracpred = 'model12_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model12_allcorr_fracpred)
                
                finame_fracpred = 'model1_allfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model1_all_fracpred)
                finame_fracpred = 'model2_allfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model2_all_fracpred)
                finame_fracpred = 'model12_allfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model12_all_fracpred)
                
                finame_fracpred = 'model1_conffracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model1_conf_fracpred)
                finame_fracpred = 'model2_conffracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model2_conf_fracpred)
                finame_fracpred = 'model12_conffracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model12_conf_fracpred)
                
                finame_fracpred = 'model1_confcorrfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model1_confcorr_fracpred)
                finame_fracpred = 'model2_confcorrfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model2_confcorr_fracpred)
                finame_fracpred = 'model12_confcorrfracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model12_confcorr_fracpred)


LEAD: 7
AVG: 2


2023-11-14 12:45:12.785280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


361/361 [==============================] - 1s 1ms/step


## Confident and Correct Fraction of Predictions, split by prediction sign

In [3]:
# model contribution fractions split by prediction sign

for l in LEADS:
    for a in AVGS:
        print('LEAD: '+str(l)+'\nAVG: '+str(a))
        
        ddir_check = '/glade/derecho/scratch/kjmayer/DATA/ENSOvsMJO/data/'
        finame_check = 'model1_confcorrfracpred_bysign_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
        if not os.path.isfile(ddir_check+finame_check):
            X1test, X2test, Ytest = get_testing(N_z500runmean=a,
                                                LEAD=l)

            INPUT_SHAPE1 = np.shape(X1test)[1:][0]
            INPUT_SHAPE2 = np.shape(X2test)[1:][0]

            model1_confcorr_fracpred = np.zeros(shape=(2,len(SEEDS)))
            model2_confcorr_fracpred = np.zeros(shape=(2,len(SEEDS)))
            model12_confcorr_fracpred = np.zeros(shape=(2,len(SEEDS)))

            for s in SEEDS:
                # ENSO MODEL
                model1, input1 = build_model(s,
                                             DROPOUT_RATE,
                                             RIDGE1,
                                             HIDDENS1,
                                             INPUT_SHAPE1,
                                             MODELNAME1)
                # MJO MODEL
                model2, input2 = build_model(s,
                                             DROPOUT_RATE,
                                             RIDGE2,
                                             HIDDENS2,
                                             INPUT_SHAPE2,
                                             MODELNAME2)   
                # COMBINE ENSO & MJO MODEL
                model = fullmodel(model1, model2,
                                  input1, input2,
                                  s)

                MODEL_FINAME = 'LEAD_'+str(l)+'_AVG_'+str(a)+'__0000'+str(s)+'.h5'
                model.load_weights(MODEL_DIR+MODEL_FINAME)

                model_rawpreds = model.predict((X1test,X2test))                

                conf    = np.max(model_rawpreds,axis=-1)
                predval = np.argmax(model_rawpreds,axis=-1)

                # ------- confident predictions --------------------------------------------------------
                per = 80
                conf_thresh = np.percentile(conf,q=per)
                # -------- confident [i_conf_predval] --------
                i_conf_predval = np.where(conf > conf_thresh)[0]

                # ----- model contribution: ------------------------------------------------------------
                model1_rawpreds, model2_rawpreds = getoutputvecs(model,
                                                                   model1,
                                                                   model2,
                                                                   X1test,
                                                                   X2test)
                # model X winning class (model X output * weight)
                model1pred = np.argmax(model1_rawpreds,axis=1)
                model2pred = np.argmax(model2_rawpreds,axis=1)

                # --------------------------------------------------------------------------------------
                # --------- confident & correct predictions contribution:-------------------------------
                # --------------------------------------------------------------------------------------

                # Where ENSO/MJO/final model (model 1/model 2/total) are correct
                i_model1_confcorr = model1pred[i_conf_predval]==Ytest[i_conf_predval]
                i_model2_confcorr = model2pred[i_conf_predval]==Ytest[i_conf_predval]
                i_model_confcorr  = predval[i_conf_predval]==Ytest[i_conf_predval]


                # terminology: "win" = modelX prediction is also (correct) full model prediction
                # model X correct & model correct (model ~X not correct)
                i_model1win = i_model1_confcorr & i_model_confcorr & ~i_model2_confcorr
                i_model2win = i_model2_confcorr & i_model_confcorr & ~i_model1_confcorr
                # model 1&2 correct & model correct
                i_model12win = i_model1_confcorr & i_model2_confcorr & i_model_confcorr



                # For correct predictions: model(X) values & predicted class when also full model prediction
                n_model1win_predval_neg = np.shape(np.where(model1pred[i_conf_predval][i_model1win] == 0)[0])[0]
                n_model1win_predval_pos = np.shape(np.where(model1pred[i_conf_predval][i_model1win] == 1)[0])[0]
                # -
                n_model2win_predval_neg = np.shape(np.where(model2pred[i_conf_predval][i_model2win] == 0)[0])[0]
                n_model2win_predval_pos = np.shape(np.where(model2pred[i_conf_predval][i_model2win] == 1)[0])[0]
                # -
                n_model12win_predval_neg = np.shape(np.where(predval[i_conf_predval][i_model12win] == 0)[0])[0]
                n_model12win_predval_pos = np.shape(np.where(predval[i_conf_predval][i_model12win] == 1)[0])[0]

                if n_model1win_predval_neg + n_model1win_predval_pos + n_model2win_predval_neg + n_model2win_predval_pos + n_model12win_predval_neg + n_model12win_predval_pos == predval[i_conf_predval][i_model_confcorr].shape[0]:
                    # print('SEED: '+str(s))
                    # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
                    model1_confcorr_fracpred[0,s-1]  = (n_model1win_predval_neg/predval[i_conf_predval][i_model_confcorr].shape[0])
                    model1_confcorr_fracpred[1,s-1]  = (n_model1win_predval_pos/predval[i_conf_predval][i_model_confcorr].shape[0])
                    # -
                    model2_confcorr_fracpred[0,s-1]  = (n_model2win_predval_neg/predval[i_conf_predval][i_model_confcorr].shape[0])
                    model2_confcorr_fracpred[1,s-1]  = (n_model2win_predval_pos/predval[i_conf_predval][i_model_confcorr].shape[0])
                    # -
                    model12_confcorr_fracpred[0,s-1] = (n_model12win_predval_neg/predval[i_conf_predval][i_model_confcorr].shape[0])
                    model12_confcorr_fracpred[1,s-1] = (n_model12win_predval_pos/predval[i_conf_predval][i_model_confcorr].shape[0])

            if SAVE:
                print('saving')
                ddir_save = '/glade/derecho/scratch/kjmayer/DATA/ENSOvsMJO/data/'

                finame_fracpred = 'model1_confcorrfracpred_bysign_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model1_confcorr_fracpred)
                finame_fracpred = 'model2_confcorrfracpred_bysign_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model2_confcorr_fracpred)
                finame_fracpred = 'model12_confcorrfracpred_bysign_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
                np.save(ddir_save+finame_fracpred, model12_confcorr_fracpred)


LEAD: 7
AVG: 2
LEAD: 7
AVG: 3
LEAD: 7
AVG: 4
LEAD: 7
AVG: 5
LEAD: 7
AVG: 6
LEAD: 7
AVG: 7
LEAD: 7
AVG: 8
LEAD: 7
AVG: 9
LEAD: 7
AVG: 10
LEAD: 7
AVG: 11
LEAD: 7
AVG: 12
LEAD: 7
AVG: 13
LEAD: 7
AVG: 14
LEAD: 7
AVG: 15
LEAD: 7
AVG: 16
LEAD: 7
AVG: 17
LEAD: 7
AVG: 18
LEAD: 7
AVG: 19
LEAD: 7
AVG: 20
LEAD: 7
AVG: 21
LEAD: 7
AVG: 22
LEAD: 7
AVG: 23
LEAD: 7
AVG: 24
LEAD: 7
AVG: 25
LEAD: 7
AVG: 26
LEAD: 7
AVG: 27
LEAD: 7
AVG: 28
LEAD: 7
AVG: 29
LEAD: 7
AVG: 30
LEAD: 7
AVG: 31
LEAD: 8
AVG: 2
LEAD: 8
AVG: 3
LEAD: 8
AVG: 4
LEAD: 8
AVG: 5
LEAD: 8
AVG: 6
LEAD: 8
AVG: 7
LEAD: 8
AVG: 8
LEAD: 8
AVG: 9
LEAD: 8
AVG: 10
LEAD: 8
AVG: 11
LEAD: 8
AVG: 12
LEAD: 8
AVG: 13
LEAD: 8
AVG: 14
LEAD: 8
AVG: 15
LEAD: 8
AVG: 16
LEAD: 8
AVG: 17
LEAD: 8
AVG: 18
LEAD: 8
AVG: 19
LEAD: 8
AVG: 20
LEAD: 8
AVG: 21
LEAD: 8
AVG: 22
LEAD: 8
AVG: 23
LEAD: 8
AVG: 24
LEAD: 8
AVG: 25
LEAD: 8
AVG: 26
LEAD: 8
AVG: 27
LEAD: 8
AVG: 28
LEAD: 8
AVG: 29
LEAD: 8
AVG: 30
LEAD: 8
AVG: 31
LEAD: 9
AVG: 2
LEAD: 9
AVG: 3
LEAD: 9
AVG: 4
LEAD: 9
AVG

2023-11-16 09:22:13.391370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


361/361 [==============================] - 1s 4ms/step
saving
LEAD: 30
AVG: 30
360/360 [==============================] - 1s 3ms/step
saving
LEAD: 30
AVG: 31
359/359 [==============================] - 1s 3ms/step
saving


### Old versions

In [5]:
# # model contribution fraction:
# #    all predictions (not just correct)
# #    confident predictions
# #    correct & confident predictions

# for l in LEADS:
#     print('LEAD: '+str(l))
#     for a in AVGS:
#         print('AVG: '+str(a))
        
#         #check if files already exist:
#         ddir_save = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/'
#         finame_confvsacc = 'model12_allfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#         if not os.path.isfile(ddir_save+finame_confvsacc):
#             print('load testing data')
#             X1test, X2test, Ytest = get_testing(N_z500runmean=a,
#                                                 LEAD=l)

#             INPUT_SHAPE1 = np.shape(X1test)[1:][0]
#             INPUT_SHAPE2 = np.shape(X2test)[1:][0]

#             model1_all_fracpred = np.zeros(shape=(len(SEEDS)))
#             model2_all_fracpred = np.zeros(shape=(len(SEEDS)))
#             model12_all_fracpred = np.zeros(shape=(len(SEEDS)))
            
#             model1_conf_fracpred = np.zeros(shape=(len(SEEDS)))
#             model2_conf_fracpred = np.zeros(shape=(len(SEEDS)))
#             model12_conf_fracpred = np.zeros(shape=(len(SEEDS)))
            
#             model1_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))
#             model2_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))
#             model12_confcorr_fracpred = np.zeros(shape=(len(SEEDS)))

#             for s in SEEDS:
#                 # ENSO MODEL
#                 model1, input1 = build_model(s,
#                                              DROPOUT_RATE,
#                                              RIDGE1,
#                                              HIDDENS1,
#                                              INPUT_SHAPE1,
#                                              MODELNAME1)
#                 # MJO MODEL
#                 model2, input2 = build_model(s,
#                                              DROPOUT_RATE,
#                                              RIDGE2,
#                                              HIDDENS2,
#                                              INPUT_SHAPE2,
#                                              MODELNAME2)   
#                 # COMBINE ENSO & MJO MODEL
#                 model = fullmodel(model1, model2,
#                                   input1, input2,
#                                   s)

#                 MODEL_FINAME = 'LEAD_'+str(l)+'_AVG_'+str(a)+'__0000'+str(s)+'.h5'
#                 model.load_weights(MODEL_DIR+MODEL_FINAME)

#                 model_rawpreds = model.predict((X1test,X2test))
#                 conf    = np.max(model_rawpreds,axis=-1)
#                 predval = np.argmax(model_rawpreds,axis=-1)

                
#                 # ------- confident predictions --------------------------------------------------------
#                 per = 80
#                 conf_thresh = np.percentile(conf,q=per)
#                 # -------- confident [i_conf_predval] --------
#                 i_conf_predval = np.where(conf > conf_thresh)[0]
                

#                 # ----- model contribution: ------------------------------------------------------------
#                 model1_rawpreds, model2_rawpreds = getoutputvecs(model,
#                                                                  model1,
#                                                                  model2,
#                                                                  X1test,
#                                                                  X2test)
#                 # model X winning class (model X output * weight)
#                 model1pred = np.argmax(model1_rawpreds,axis=1)
#                 model2pred = np.argmax(model2_rawpreds,axis=1)

                
                
#                 # ---------- all predictions contribution:
#                 i_model1_samefinalpred = model1pred==predval
#                 i_model2_samefinalpred = model2pred==predval
                
#                 # terminology: "win" = modelX prediction is also full model prediction
#                 # model X same as final prediction & model ~X does not have that prediction
#                 i_model1win = i_model1_samefinalpred & ~i_model2_samefinalpred
#                 i_model2win = i_model2_samefinalpred & ~i_model1_samefinalpred
#                 # model 1&2 have same final prediction
#                 i_model12win = i_model1_samefinalpred & i_model2_samefinalpred
                
                
#                 # number of predictions of model(X) predicted class that is also full model prediction
#                 n_model1win_predval = model1pred[i_model1win].shape[0]
#                 n_model2win_predval = model2pred[i_model2win].shape[0]
#                 n_model12win_predval = predval[i_model12win].shape[0]
                
#                 # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
#                 if predval.shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
#                     print('SEED: '+str(s))
#                     # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
#                     model1_all_fracpred[s-1]  = (n_model1win_predval/predval.shape[0])
#                     model2_all_fracpred[s-1]  = (n_model2win_predval/predval.shape[0])
#                     model12_all_fracpred[s-1] = (n_model12win_predval/predval.shape[0]) 
                    
                    
                
#                 # --------- confident predictions contribution:
#                 i_model1_samefinalconfpred = model1pred[i_conf_predval]==predval[i_conf_predval]
#                 i_model2_samefinalconfpred = model2pred[i_conf_predval]==predval[i_conf_predval]
                
#                 # terminology: "win" = modelX prediction is also full model prediction
#                 # model X same as final prediction & model ~X does not have that prediction
#                 i_model1win = i_model1_samefinalconfpred & ~i_model2_samefinalconfpred
#                 i_model2win = i_model2_samefinalconfpred & ~i_model1_samefinalconfpred
#                 # model 1&2 have same final prediction
#                 i_model12win = i_model1_samefinalconfpred & i_model2_samefinalconfpred
                
                
#                 # number of predictions of model(X) predicted class that is also full model prediction
#                 n_model1win_predval = model1pred[i_conf_predval][i_model1win].shape[0]
#                 n_model2win_predval = model2pred[i_conf_predval][i_model2win].shape[0]
#                 n_model12win_predval = predval[i_conf_predval][i_model12win].shape[0]

                
#                 # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
#                 if predval[i_conf_predval].shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
#                     print('SEED: '+str(s))
#                     # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
#                     model1_conf_fracpred[s-1]  = (n_model1win_predval/predval[i_conf_predval].shape[0])
#                     model2_conf_fracpred[s-1]  = (n_model2win_predval/predval[i_conf_predval].shape[0])
#                     model12_conf_fracpred[s-1] = (n_model12win_predval/predval[i_conf_predval].shape[0]) 
                
                
                
#                 # --------- confident & correct predictions contribution:
#                 # Where ENSO/MJO/final model (model 1/model 2/total) are correct
#                 i_model1_confcorr = model1pred[i_conf_predval]==Ytest[i_conf_predval]
#                 i_model2_confcorr = model2pred[i_conf_predval]==Ytest[i_conf_predval]
#                 i_model_confcorr  = predval[i_conf_predval]==Ytest[i_conf_predval]

#                 # terminology: "win" = modelX prediction is also (correct) full model prediction
#                 # model X correct & model correct (model ~X not correct)
#                 i_model1win = i_model1_confcorr & i_model_confcorr & ~i_model2_confcorr
#                 i_model2win = i_model2_confcorr & i_model_confcorr & ~i_model1_confcorr
#                 # model 1&2 correct & model correct
#                 i_model12win = i_model1_confcorr & i_model2_confcorr & i_model_confcorr

#                 # For correct predictions: model(X) values & predicted class when also full model prediction
#                 n_model1win_predval = model1pred[i_conf_predval][i_model1win].shape[0]
#                 n_model2win_predval = model2pred[i_conf_predval][i_model2win].shape[0]
#                 n_model12win_predval = predval[i_conf_predval][i_model12win].shape[0]
                
                
#                 # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
#                 if predval[i_conf_predval][i_model_confcorr].shape[0] == n_model1win_predval + n_model2win_predval + n_model12win_predval:
#                     print('SEED: '+str(s))
#                     # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
#                     model1_confcorr_fracpred[s-1]  = (n_model1win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])
#                     model2_confcorr_fracpred[s-1]  = (n_model2win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])
#                     model12_confcorr_fracpred[s-1] = (n_model12win_predval/predval[i_conf_predval][i_model_confcorr].shape[0])    
            
            
#             # ------------------------------------------------------------------------------------------
#             if SAVE: 
#                 print('saving')
#                 ddir_save = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/'

#                 finame_fracpred = 'model1_allfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model1_all_fracpred)
#                 finame_fracpred = 'model2_allfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model2_all_fracpred)
#                 finame_fracpred = 'model12_allfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model12_all_fracpred)
                
#                 finame_fracpred = 'model1_conffracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model1_conf_fracpred)
#                 finame_fracpred = 'model2_conffracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model2_conf_fracpred)
#                 finame_fracpred = 'model12_conffracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model12_conf_fracpred)
                
#                 finame_fracpred = 'model1_confcorrfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model1_confcorr_fracpred)
#                 finame_fracpred = 'model2_confcorrfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model2_confcorr_fracpred)
#                 finame_fracpred = 'model12_confcorrfracpredv2_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model12_confcorr_fracpred)


In [7]:
# # model raw predictions
# # confidence vs accuracy
# # model contribution fraction (correct)
# for l in LEADS:
#     print('LEAD: '+str(l))
#     for a in AVGS:
#         print('AVG: '+str(a))
        
#         #check if files already exist:
#         ddir_save = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/'
#         finame_confvsacc = 'confvsacc_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#         if not os.path.isfile(ddir_save+finame_confvsacc):
#             print('load testing data')
#             X1test, X2test, Ytest = get_testing(N_z500runmean=a,
#                                                 LEAD=l)

#             INPUT_SHAPE1 = np.shape(X1test)[1:][0]
#             INPUT_SHAPE2 = np.shape(X2test)[1:][0]

#             confvsacc = np.zeros(shape=(len(SEEDS),100))

#             model1_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))
#             model2_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))
#             model_rawpreds = np.zeros(shape=(len(SEEDS),np.shape(X1test)[0],2))

#             model1_fracpred = np.zeros(shape=(len(SEEDS)))
#             model2_fracpred = np.zeros(shape=(len(SEEDS)))
#             model12_fracpred = np.zeros(shape=(len(SEEDS)))

#             for s in SEEDS:
#                 # ENSO MODEL
#                 model1, input1 = build_model(s,
#                                              DROPOUT_RATE,
#                                              RIDGE1,
#                                              HIDDENS1,
#                                              INPUT_SHAPE1,
#                                              MODELNAME1)
#                 # MJO MODEL
#                 model2, input2 = build_model(s,
#                                              DROPOUT_RATE,
#                                              RIDGE2,
#                                              HIDDENS2,
#                                              INPUT_SHAPE2,
#                                              MODELNAME2)   
#                 # COMBINE ENSO & MJO MODEL
#                 model = fullmodel(model1, model2,
#                                   input1, input2,
#                                   s)

#                 MODEL_FINAME = 'LEAD_'+str(l)+'_AVG_'+str(a)+'__0000'+str(s)+'.h5'
#                 model.load_weights(MODEL_DIR+MODEL_FINAME)

#                 model_rawpreds[s-1] = model.predict((X1test,X2test))
#                 conf    = np.max(model_rawpreds[s-1],axis=-1)
#                 predval = np.argmax(model_rawpreds[s-1],axis=-1)

#                 # ----- confidence vs accuracy for all seeds:
#                 confvsacc[s-1], _, _ = confvacc(confval = conf,
#                                               predval = predval,
#                                               Ytest   = Ytest)

#                 # ----- model contribution:
#                 model1_rawpreds[s-1], model2_rawpreds[s-1] = getoutputvecs(model,
#                                                                        model1,
#                                                                        model2,
#                                                                        X1test,
#                                                                        X2test)
#                 # model X winning class (model X output * weight)
#                 model1pred = np.argmax(model1_rawpreds[s-1],axis=1)
#                 model2pred = np.argmax(model2_rawpreds[s-1],axis=1)

#                 # Where ENSO/MJO/final model (model 1/model 2/total) are correct
#                 i_model1corr = model1pred==Ytest
#                 i_model2corr = model2pred==Ytest
#                 i_modelcorr  = predval==Ytest

#                 # terminology: "win" = modelX prediction is also (correct) full model prediction
#                 # model X correct & model correct (model ~X not correct)
#                 i_model1win = i_model1corr & i_modelcorr & ~i_model2corr
#                 i_model2win = i_model2corr & i_modelcorr & ~i_model1corr
#                 # model 1&2 correct & model correct
#                 i_model12win = i_model1corr & i_model2corr & i_modelcorr

#                 # For correct predictions: model(X) values & predicted class when also full model prediction
#                 model1win_contribution = model1_rawpreds[s-1][i_model1win]
#                 model1win_predval = model1pred[i_model1win]

#                 model2win_contribution = model2_rawpreds[s-1][i_model2win]
#                 model2win_predval = model2pred[i_model2win]

#                 model12win_predval = model_rawpreds[s-1][i_model12win]

#                 # these shapes should be equal (True), assuming ~i_model1corr & ~i_model2corr & i_modelcorr doesnt happen
#                 if model_rawpreds[s-1][i_modelcorr].shape[0] == model1win_predval.shape[0] + model2win_predval.shape[0] + model12win_predval.shape[0]:
#                     print('SEED: '+str(s))
#                     # Percentage of model predictions correct due to just ENSO/MJO/ENSO&MJO:
#                     model1_fracpred[s-1]  = (model1win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])
#                     model2_fracpred[s-1]  = (model2win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])
#                     model12_fracpred[s-1] = (model12win_predval.shape[0]/model_rawpreds[s-1][i_modelcorr].shape[0])        

#             if SAVE:
#                 print('saving')
#                 ddir_save = '/glade/work/kjmayer/research/catalyst/ENSOvsMJO/data/'
                
#                 finame_confvsacc = 'confvsacc_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_confvsacc, confvsacc)

#                 finame_rawpred = 'model1_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_rawpred, model1_rawpreds)
#                 finame_rawpred = 'model2_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_rawpred, model2_rawpreds)
#                 finame_rawpred = 'model_rawpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_rawpred, model_rawpreds)

#                 finame_fracpred = 'model1_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model1_fracpred)
#                 finame_fracpred = 'model2_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model2_fracpred)
#                 finame_fracpred = 'model12_fracpred_LEAD_'+str(l)+'_AVG_'+str(a)+'__00001-00005.npy'
#                 np.save(ddir_save+finame_fracpred, model12_fracpred)
